# Biostat 257 Homework 3

**Due May 6 @ 11:59PM**

In [2]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, haswell)


In [7]:
#import Pkg; Pkg.add("Distributions")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed PDMats ──────── v0.11.8
   Installed StatsFuns ───── v0.9.18
   Installed Distributions ─ v0.25.56
    Updating `~/biostat-257-2022-spring/Project.toml`
  [31c24e10] + Distributions v0.25.56
    Updating `~/biostat-257-2022-spring/Manifest.toml`
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.56
  [1a297f60] + FillArrays v0.13.2
  [90014a1f] + PDMats v0.11.8
  [1fd47b50] + QuadGK v2.4.2
  [79098fc4] + Rmath v0.7.0
  [4c63d2b9] + StatsFuns v0.9.18
  [f50d1b31] + Rmath_jll v0.3.0+0
  [4607b0f0] + SuiteSparse
Precompiling project...
  ✓ DensityInterface
  ✓ PDMats
  ✓ Rmath_jll
  ✓ QuadGK
  ✓ FillArrays
  ✓ Rmath
  ✓ StatsFuns
  ✓ Distributions
  8 dependencies successfully precompiled in 17 seconds (220 already precompiled)


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

#### Solution

$$E[\mathbf{Y}_i] = \mathbf{X}_i \boldsymbol{\beta};$$ $$var(\mathbf{Y}_i) = \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 \mathbf{I}_{n_i} = \mathbf{V}_i$$

Therefore, $\mathbf{Y}_i \sim N_{n_i}(\mathbf{X}_i \boldsymbol{\beta}, \mathbf{V}_i)$

$L(\mathbf{Y}_i, \mathbf{X}_i, \mathbf{Z}_i| \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \frac{1}{(2 \pi)^{n_i/2} (\det{\mathbf{V}_i})^{1/2}} \exp({-\frac{1}{2}(\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})})$

$l(\mathbf{Y}_i, \mathbf{X}_i, \mathbf{Z}_i| \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = -\frac{n_i}{2} \log (2 \pi) -\frac{1}{2} \log(\det{\mathbf{V}_i}) -\frac{1}{2} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{Y}_i - \mathbf{X}_i \boldsymbol{\beta})$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [3]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2)) # stores px1
    storage_q  = Vector{T}(undef, size(Z, 2)) # stores qx1
    xtx        = transpose(X) * X # stores X'X
    ztx        = transpose(Z) * X # Z'X
    ztz        = transpose(Z) * Z # stores Z'Z
    storage_qq = similar(ztz) # stores Sigma
    LmmObs(y, X, Z, storage_p, storage_q, xtx, ztx, ztz, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

**Step 1: Prototype**

In [59]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood
    c = - (n / 2) * log(2π) - 0.5 * log(n * σ²) # constant, should not allocate
    sum1 = det(I + (L * L') * obs.ztz / σ²) # determinant
    #sum1 = σ² * det(I + obs.Z * (L * L') * transpose(obs.Z) / σ²)
    sum2 = transpose(obs.y) * (obs.y) - 2transpose(obs.y) * obs.X * β + transpose(β) * obs.xtx * β
    star = inv(inv(L * L') + obs.ztz / σ²)
    sum3 = transpose(obs.y) * obs.Z * star * transpose(obs.Z) * obs.y - 2transpose(obs.y) * obs.Z * star * obs.ztx * β + 
        transpose(β) * transpose(obs.ztx) * star * obs.ztx * β
    loglik = c - 0.5 * log(sum1) - 0.5 * sum2 / σ² + 0.5 * sum3 / (σ²)^2
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return loglik
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [50]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [5.0e-324, 5.0e-324, 2.521879663e-314, 2.521879695e-314, 0.0], [2.528598703e-314, 2.528598719e-314, 2.9130523043e-314], [2000.0 -16.87094

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [52]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [60]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-2854.7174114814857

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [45]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

LoadError: AssertionError: logl!(obs, β, Matrix((cholesky(Σ)).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [ ]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

In [ ]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [ ]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [ ]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.